In [ ]:
import requests
from plyer import notification
import schedule
import time
import openpyxl
import datetime

In [ ]:
# Replace 'YOUR_API_KEY_HERE' with your actual API key from https://www.football-data.org/
api_key = 'YOUR_API_KEY_HERE'  
headers = {'X-Auth-Token': api_key}
url = "http://api.football-data.org/v2/matches"

In [ ]:
def fetch_scores():
    response = requests.get(url, headers=headers)
    try:
        data = response.json()
        # Check if the 'matches' key exists in the response
        if 'matches' in data and len(data['matches']) > 0:
            match = data['matches'][0]
            score_line = f"{match['score']['fullTime']['homeTeam']} - {match['score']['fullTime']['awayTeam']}"
            team_home = match['homeTeam']['name']
            team_away = match['awayTeam']['name']
            return f"{team_home} {score_line} {team_away}"
        else:
            return "No matches currently available."
    except Exception as e:
        return f"Error fetching data: {e}"

In [ ]:
def send_notification():
    match_score = fetch_scores()
    notification.notify(
        title='Live Sports Score Update',
        message=match_score,
        app_icon=None,  # You can add a path to an app icon here
        timeout=10,  # Notification will stay for 10 seconds
    )
    # Record data into Excel
    record_to_excel(match_score)

In [ ]:
def record_to_excel(match_data):
    """Records the match data to an Excel spreadsheet."""
    try:
        workbook = openpyxl.load_workbook('sports_data.xlsx')
        sheet = workbook.active
    except FileNotFoundError:
        workbook = openpyxl.Workbook()
        sheet = workbook.active
        sheet.append(["Home Team", "Score", "Away Team", "Date"])

    # Extract data (assuming match_data is in the format "HomeTeam Score - Score AwayTeam")
    try:
        home_team, score, away_team = match_data.split(' ')
        today = datetime.date.today().strftime("%Y-%m-%d")
        sheet.append([home_team, score, away_team, today])  # Appending match data and today's date
    except ValueError:
        today = datetime.date.today().strftime("%Y-%m-%d")
        sheet.append(["Error processing data", "", "", today])

    workbook.save('sports_data.xlsx')

In [ ]:
# Test sending a notification
send_notification()

In [ ]:
schedule.every().day.at("08:00").do(send_notification)

# Keep the script running to trigger the scheduled tasks
while True:
    schedule.run_pending()
    time.sleep(1)